# Experiment

## Imports

In [1]:
from Environments import (CartPole, Highway, Hopper, LunarLander,
                          Swimmer)
from LLM.LLMOptions import llm_options
from log.log_config import init_logger
from VIRAL import VIRAL
init_logger("DEBUG")

In [2]:
def runs(
    total_timesteps: int,
    nb_vec_envs: int,
    nb_refined: int,
    human_feedback: bool,
    video_description: bool,
    legacy_training: bool,
    actor_model: str,
    critic_model: str,
    env: str,
    observation_space: str,
    goal: str,
    image: str,
    nb_gen: int,
    nb_runs: int,
    proxies: dict,
    focus: str = "",
):
    """help wrapper for launch several runs

    Args:
        total_timesteps (int): 
        nb_vec_envs (int): 
        nb_refined (int): 
        human_feedback (bool): 
        video_description (bool): 
        legacy_training (bool): 
        actor_model (str): 
        critic_model (str): 
        env (str): 
        observation_space (str): 
        goal (str): 
        image (str): 
        nb_gen (int): 
        nb_runs (int): 
        proxies (dict): 
        focus (str, optional): . Defaults to "".
    """
    switcher = {
        "Cartpole": CartPole,
        "LunarLander": LunarLander,
        "Highway": Highway,
        "Swimmer": Swimmer,
        "Hopper": Hopper,
    }
    instance = switcher[env]()
    if observation_space != "":
        instance.prompt["Observation Space"] = observation_space
    if goal is not None:
        instance.prompt["Goal"] = goal
    else:
        instance.prompt.pop("Goal", None)
    if image is not None:
        instance.prompt["Image"] = image
    else:
        instance.prompt.pop("Image", None)
    def run():
        viral = VIRAL(
            env_type=instance,
            model_actor=actor_model,
            model_critic=critic_model,
            hf=human_feedback,
            vd=video_description,
            nb_vec_envs=nb_vec_envs,
            options=llm_options,
            legacy_training=legacy_training,
            training_time=total_timesteps,
            proxies=proxies,
        )
        viral.generate_context()
        viral.generate_reward_function(nb_gen, nb_refined, focus)
        viral.policy_trainer.start_vd(viral.memory[1].policy, 1)

    for r in range(nb_runs):
        print(f"#######  {r}  ########")
        run()

In [3]:
proxies = { 
	"http"  : "socks5h://localhost:1080", 
	"https" : "socks5h://localhost:1080", 
}

## LunarLander

In [4]:
obs_space = """Box([ -2.5 -2.5 -10. -10. -6.2831855 -10. -0. -0. ], 
[ 2.5 2.5 10. 10. 6.2831855 10. 1. 1. ], (8,), float32)
The state is an 8-dimensional vector: 
the coordinates of the lander in x & y, 
its linear velocities in x & y, 
its angle, its angular velocity, 
and two booleans that represent whether each leg is in contact with the ground or not.
"""
goal = "Do not land but do not crash, i want a stationary Flight"
image = 'Environments/img/stationary_flight.png'

### Only text

In [5]:
runs(
    total_timesteps=30_000,
    nb_vec_envs=1,
    nb_refined=0,
    human_feedback=False,
    video_description=False,
    legacy_training=False,
    actor_model="qwen2.5-coder:32b",
    critic_model="llama3.2-vision",
    env="LunarLander",
    observation_space=obs_space,
    goal=goal,
    image=None,
    nb_gen=1,
    nb_runs=10,
    proxies=proxies,
)


22:09:21 OllamaChat.py:39 INFO
	System: 
        You're a reinforcement learning expert specializing in the design of python reward functions.
        Strict criteria:
        1. Take care of Generate ALWAYS DIFFERENTS reward function per Response iteration
        2. Complete ONLY the reward function code
        3. Give no additional explanations
        4. STOP immediately your completion after the last return
        5. Assuming Numpy already imported as np
        6. Take into the observation of the state, the is_success boolean flag, the is_failure boolean flag
        , Options: {'temperature': 0.9, 'seed': 237834}

22:09:21 OllamaChat.py:39 INFO
	System: 
        You're a reinforcement learning expert, great mathematician and physician.
        As a critic, you're going to explains step by step, the environment.
        Every answer you make will be contained in an xml <HELP> tag., Options: {'temperature': 0.9, 'seed': 237834}


#######  0  ########



22:09:27 OllamaChat.py:195 INFO
	Response -1:


<HELP>
The given observation space describes an 8-dimensional vector representing the state of the lander in a 2D environment. The vector is composed of:

* `x` and `y` coordinates of the lander: These are the positions of the lander on the x and y axes, ranging from `-2.5` to `2.5`.

* `vx` and `vy`: These represent the linear velocities of the lander in the x and y directions, respectively.

* `theta`: This is the angle of the lander with respect to the horizontal plane, which can range from `-6.2831855` (π radians) to `10`. Note that this range seems inconsistent; typically, angles would be defined within a single period of 2π or π radians for simplicity.

* `omega`: This represents the angular velocity of the lander around its vertical axis.

* `leg1_contact` and `leg2_contact`: These are two boolean values indicating whether each leg is in contact with the ground or not.

To achieve a stationary flight, meaning to hover without crashing but also preventing landing (i.e., maintaini


22:11:43 OllamaChat.py:195 INFO
	Response 0:


...


22:24:24 OllamaChat.py:195 INFO
	Response 0:


def reward_func(observations:np.ndarray, is_success:bool, is_failure:bool) -> float:
    vx, vy, _, _, _, _, leg1_contact, leg2_contact = observations
    velocity_penalty = abs(vx) + abs(vy)
    contact_penalty = -50 * (leg1_contact or leg2_contact)
    reward = 1 - velocity_penalty / 20 + contact_penalty if not is_failure else -100
    return reward


22:24:29 GenCode.py:228 DEBUG
	Reward function output: 0.9296165585517884

22:24:29 PolicyTrainer.py:60 INFO
	state 1 begin is learning

22:24:29 PolicyTrainer.py:319 DEBUG
	simple env


Output()


22:25:20 PolicyTrainer.py:80 INFO
	state 1 has finished learning with performances: 0.0

22:25:20 PolicyTrainer.py:152 INFO
	the threshold is 0.9
/home/valentin/.conda/envs/llm/lib/python3.11/site-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /home/valentin/Travail/VIRAL/src/records/LunarLander-v3 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(

22:25:21 VIRAL.py:217 INFO
	video safe at: records/LunarLander-v3/LunarLander-v3_187191-last.mp4
/home/valentin/.conda/envs/llm/lib/python3.11/site-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /home/valentin/Travail/VIRAL/src/records/LunarLander-v3 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


### Only Image

In [5]:
runs(
    total_timesteps=30_000,
    nb_vec_envs=1,
    nb_refined=0,
    human_feedback=False,
    video_description=False,
    legacy_training=False,
    actor_model="qwen2.5-coder:32b",
    critic_model="llama3.2-vision",
    env="LunarLander",
    observation_space=obs_space,
    goal=None,
    image=image,
    nb_gen=1,
    nb_runs=10,
    proxies=proxies,
)


20:23:38 OllamaChat.py:39 INFO
	System: 
        You're a reinforcement learning expert specializing in the design of python reward functions.
        Strict criteria:
        1. Take care of Generate ALWAYS DIFFERENTS reward function per Response iteration
        2. Complete ONLY the reward function code
        3. Give no additional explanations
        4. STOP immediately your completion after the last return
        5. Assuming Numpy already imported as np
        6. Take into the observation of the state, the is_success boolean flag, the is_failure boolean flag
        , Options: {'temperature': 0.9, 'seed': 319551}

20:23:38 OllamaChat.py:39 INFO
	System: 
        You're a reinforcement learning expert, great mathematician and physician.
        As a critic, you're going to explains step by step, the environment.
        Every answer you make will be contained in an xml <HELP> tag., Options: {'temperature': 0.9, 'seed': 319551}


#######  0  ########



20:23:56 OllamaChat.py:195 INFO
	Response -1:


<HELP>

1. **Goal and Annotation:** The red annotation indicates the goal of the game or simulation, which is to land on the ground safely.

2. **Meaning:** The image appears to be a screenshot from a video game or simulation where an agent (lander) needs to navigate through space and land on a planet's surface without crashing. The red line marks the boundary between the air and the ground, serving as a visual indicator of the landing zone. 

3. **Agent Goal:** The agent must successfully navigate to the designated landing site within the marked area and come to rest without any contact with the ground (to avoid crashing), thereby achieving the goal.

4. **Observation Space Description:**

   *   **Coordinates (x, y):** These are two of the first four components in the observation space vector provided at the end of the prompt.
       *   They represent the current position of the lander on the x and y axes of a 2D plane or grid that models the surface it's landing on.

   *   **Linea


20:24:27 OllamaChat.py:195 INFO
	Response 0:


...


20:35:26 OllamaChat.py:195 INFO
	Response 0:


def reward_func(observations:np.ndarray, is_success:bool, is_failure:bool) -> float:
    """Reward function for LunarLander-v3

    Args:
        observations (np.ndarray): observation on the current state
        is_success (bool): True if the goal is achieved, False otherwise
        is_failure (bool): True if the episode ends unsuccessfully, False otherwise

    Returns:
        float: The reward for the current step
    """
    x_velocity = observations[2]
    y_velocity = observations[3]
    angle = observations[4]
    angular_velocity = observations[5]

    stationary_reward = -abs(x_velocity) - abs(y_velocity) - abs(angle) - abs(angular_velocity)
    success_bonus = 100 if is_success else 0
    failure_penalty = -100 if is_failure else 0

    return stationary_reward + success_bonus + failure_penalty


20:35:34 GenCode.py:228 DEBUG
	Reward function output: -1.566845178604126

20:35:34 PolicyTrainer.py:60 INFO
	state 1 begin is learning

20:35:34 PolicyTrainer.py:319 DEBUG
	simple env


Output()


20:36:32 PolicyTrainer.py:80 INFO
	state 1 has finished learning with performances: 0.0

20:36:32 PolicyTrainer.py:152 INFO
	the threshold is 0.9
/home/valentin/.conda/envs/llm/lib/python3.11/site-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /home/valentin/Travail/VIRAL/src/records/LunarLander-v3 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(

20:36:37 VIRAL.py:217 INFO
	video safe at: records/LunarLander-v3/LunarLander-v3_795773-last.mp4
/home/valentin/.conda/envs/llm/lib/python3.11/site-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /home/valentin/Travail/VIRAL/src/records/LunarLander-v3 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


### Text+Image

In [6]:
runs(
    total_timesteps=30_000,
    nb_vec_envs=1,
    nb_refined=0,
    human_feedback=False,
    video_description=False,
    legacy_training=False,
    actor_model="qwen2.5-coder:32b",
    critic_model="llama3.2-vision",
    env="LunarLander",
    observation_space=obs_space,
    goal=goal,
    image=image,
    nb_gen=1,
    nb_runs=10,
    proxies=proxies,
)


21:04:49 OllamaChat.py:39 INFO
	System: 
        You're a reinforcement learning expert specializing in the design of python reward functions.
        Strict criteria:
        1. Take care of Generate ALWAYS DIFFERENTS reward function per Response iteration
        2. Complete ONLY the reward function code
        3. Give no additional explanations
        4. STOP immediately your completion after the last return
        5. Assuming Numpy already imported as np
        6. Take into the observation of the state, the is_success boolean flag, the is_failure boolean flag
        , Options: {'temperature': 0.9, 'seed': 631760}

21:04:49 OllamaChat.py:39 INFO
	System: 
        You're a reinforcement learning expert, great mathematician and physician.
        As a critic, you're going to explains step by step, the environment.
        Every answer you make will be contained in an xml <HELP> tag., Options: {'temperature': 0.9, 'seed': 631760}


#######  0  ########



21:05:00 OllamaChat.py:195 INFO
	Response -1:


**Step 1: Understand the Environment**

The environment is a challenging task where an agent must land a spacecraft on Mars while avoiding crashing. The state space consists of eight dimensions:

*   $x$ and $y$ coordinates
*   Linear velocities in $x$ and $y$
*   Angle and angular velocity
*   Two booleans indicating whether each leg is in contact with the ground or not

**Step 2: Identify the Goal**

The goal is to achieve a stationary flight, which means the agent must navigate the spacecraft to hover in place without crashing.

**Step 3: Analyze the Observation Space**

From the observation space provided earlier:

<HELP> The observation space consists of eight dimensions, including the x and y coordinates of the lander, its linear velocities in these directions, its angle and angular velocity, and two booleans indicating whether each leg is in contact with the ground or not. This suggests that the agent has a good understanding of its spatial position and orientation within the en


21:05:16 OllamaChat.py:195 INFO
	Response 0:


...


21:17:25 OllamaChat.py:195 INFO
	Response 0:


```python
def reward_func(observations: np.ndarray, is_success: bool, is_failure: bool) -> float:
    """Reward function for LunarLander-v3

    Args:
        observations (np.ndarray): observation on the current state
        is_success (bool): True if the goal is achieved, False otherwise
        is_failure (bool): True if the episode ends unsuccessfully, False otherwise

    Returns:
        float: The reward for the current step
    """
    x_velocity = abs(observations[2])
    y_velocity = abs(observations[3])
    angle = abs(observations[4])
    angular_velocity = abs(observations[5])
    leg_contact_1 = observations[6]
    leg_contact_2 = observations[7]

    if is_success:
        return -1.0
    elif is_failure:
        return -1.0

    reward = 0.0
    reward -= x_velocity * 0.01
    reward -= y_velocity * 0.01
    reward -= angle * 0.05
    reward -= angular_velocity * 0.1
    if leg_contact_1 or leg_contact_2:
        reward -= 0.1

    return reward
```


21:17:36 GenCode.py:228 DEBUG
	Reward function output: -0.021538926512002947

21:17:36 PolicyTrainer.py:60 INFO
	state 1 begin is learning

21:17:36 PolicyTrainer.py:319 DEBUG
	simple env


Output()


21:18:30 PolicyTrainer.py:80 INFO
	state 1 has finished learning with performances: 0.0

21:18:31 PolicyTrainer.py:152 INFO
	the threshold is 0.9
/home/valentin/.conda/envs/llm/lib/python3.11/site-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /home/valentin/Travail/VIRAL/src/records/LunarLander-v3 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(

21:18:33 VIRAL.py:217 INFO
	video safe at: records/LunarLander-v3/LunarLander-v3_600632-last.mp4
/home/valentin/.conda/envs/llm/lib/python3.11/site-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /home/valentin/Travail/VIRAL/src/records/LunarLander-v3 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
